In [ ]:
import viennaps as ps

In [ ]:
# -------------------------------
# Helper function: Create domain with a simple masked trench
# -------------------------------
def createTrenchMask():
    extent = 30
    gridDelta = 0.3

    # Create a 2D simulation domain with specified extent and resolution
    domain = ps.Domain(xExtent=extent, gridDelta=gridDelta)

    # Add a rectangular trench with a flat bottom (depth=0) and mask layer on top
    ps.MakeTrench(domain, trenchWidth=10.0, trenchDepth=0.0, maskHeight=10.0, maskTaperAngle=10).apply()

    return domain

### Units

The physical plasma etching models require units to be set

In [ ]:
ps.Length.setUnit("um") # Set length unit to micrometers
ps.Time.setUnit("min") # Set time unit to minutes

### Set up model parameters

Available parameters can also be found in the [documentation](https://viennatools.github.io/ViennaPS/models/prebuilt/SF6O2Etching.html)

In [ ]:
params = ps.SF6O2Etching.defaultParameters()

params.ionFlux = 10.0
params.etchantFlux = 4500.0
params.passivationFlux = 800.0

params.Ions.meanEnergy = 100.0
params.Ions.sigmaEnergy = 10.0
params.Ions.exponent = 500 

model = ps.SF6O2Etching(params)

### Configure advection and ray tracing settings

Additional settings concerning the surface advection scheme and the coverage. See [here](https://viennatools.github.io/ViennaPS/process/) for more information.

In [ ]:
processParams = ps.AdvectionParameters()
processParams.spatialScheme = ps.SpatialScheme.LOCAL_LAX_FRIEDRICHS_1ST_ORDER

rayTracingParams = ps.RayTracingParameters()
rayTracingParams.raysPerPoint = 1000

In [ ]:
domain = createTrenchMask()
domain.saveVolumeMesh("SF6O2Etching_1") # Save the volume mesh to a file

process = ps.Process()
process.setDomain(domain)
process.setProcessModel(model)
process.setProcessDuration(10) # Set process duration to 10 minutes
process.setParameters(processParams)
process.setParameters(rayTracingParams)

### Coverage Steady-State Configurations

Before advancing the surface geometry, the surface coverages must reach a steady state. This ensures that the reaction rates derived from the local fluxes and coverages are physically meaningful and numerically stable. Convergence is determined using the coverageDeltaThreshold, which defines the minimum relative change in coverage between consecutive iterations. If the change falls below this threshold, the coverages are considered stationary. Alternatively, convergence is enforced after a fixed number of iterations defined by maxCoverageInitIterations, even if the threshold has not been met.


In [ ]:
covParams = ps.CoverageParameters()
covParams.tolerance = 1e-4
covParams.maxIterations = 10
process.setParameters(covParams)

#### Logging

Configure additional intermediate output by setting the [log level](https://viennatools.github.io/ViennaPS/misc/logging.html).

In [ ]:
 # Set log level to intermediate for additional output
ps.Logger.setLogLevel(ps.LogLevel.INTERMEDIATE)

In [ ]:
# Run the process
process.apply()

In [ ]:
domain.show()

In [ ]:
# Save the volume mesh to a file
domain.saveVolumeMesh("SF6O2Etching_2")